In [18]:
import pandas as pd
import numpy as np
import networkx as nx
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import random
import pickle
from sklearn.ensemble import RandomForestClassifier

print("Libraries imported successfully.")

# Section 2: Preprocessing and Graph Construction

# File paths
original_features_path = "../foursquare_data/features.json"
original_edges_path = "../foursquare_data/edges.csv"

# Load data
def load_data():
    print("Loading original features and edges...")
    with open(original_features_path, "r") as f:
        features = json.load(f)
    print(f"Loaded {len(features)} nodes with features.")
    
    edges = pd.read_csv(
        original_edges_path, names=["source", "target"], skiprows=1
    )  # Skip header row
    print(f"Loaded {len(edges)} edges.")
    return features, edges

# Create graph and extract largest connected component
def create_graph(features, edges):
    print("Creating graph from edges...")
    edges["source"] = edges["source"].astype(str)
    edges["target"] = edges["target"].astype(str)
    G = nx.from_pandas_edgelist(edges, source="source", target="target")
    print(f"Graph created with {len(G.nodes)} nodes and {len(G.edges)} edges.")

    print("Identifying the largest connected component...")
    largest_cc = max(nx.connected_components(G), key=len)
    G_lcc = G.subgraph(largest_cc).copy()
    print(f"Largest connected component has {len(G_lcc.nodes)} nodes and {len(G_lcc.edges)} edges.")

    print("Assigning features to nodes...")
    for node in G_lcc.nodes:
        if node in features and len(features[node]) == 8:
            G_lcc.nodes[node]["features"] = np.array(features[node], dtype=float)
    print("Node features assigned.")
    
    return G_lcc

print("Starting graph preprocessing...")
features, edges = load_data()
G = create_graph(features, edges)
print(f"Graph preprocessing complete. Final graph has {len(G.nodes)} nodes and {len(G.edges)} edges.")

def check_connectivity_bfs(G):
    print("Performing BFS to ensure all nodes are connected...")
    start_node = next(iter(G.nodes))  # Get an arbitrary starting node
    visited = set()
    queue = [start_node]

    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            queue.extend(neighbor for neighbor in G.neighbors(node) if neighbor not in visited)

    if len(visited) == len(G.nodes):
        print("All nodes are connected. The graph is a single connected component.")
    else:
        print(f"Graph is not fully connected. Only {len(visited)} out of {len(G.nodes)} nodes are reachable.")

check_connectivity_bfs(G)  # Ensure the graph is a single connected component

# Section 3A: Create Feature Vectors

def create_feature_vectors(G, edges):
    print("Creating feature vectors for ML tasks...")
    X, y = [], []

    print("Processing positive samples (existing edges)...")
    for i, (_, row) in enumerate(edges.iterrows()):
        node1, node2 = str(row["source"]), str(row["target"])
        if (
            node1 in G.nodes 
            and node2 in G.nodes 
            and "features" in G.nodes[node1] 
            and "features" in G.nodes[node2]
        ):
            feature_vector = G.nodes[node1]["features"] - G.nodes[node2]["features"]
            X.append(feature_vector)
            y.append(1)
        if i % 50000 == 0 and i > 0:
            print(f"Processed {i} positive samples.")

    print("Generating negative samples (random non-existing edges)...")
    all_nodes = [node for node in G.nodes if "features" in G.nodes[node]]  # Nodes with features
    for i in range(len(edges)):
        node1, node2 = np.random.choice(all_nodes, 2, replace=False)
        if not G.has_edge(node1, node2):
            feature_vector = G.nodes[node1]["features"] - G.nodes[node2]["features"]
            X.append(feature_vector)
            y.append(0)
        if i % 50000 == 0 and i > 0:
            print(f"Generated {i} negative samples.")

    print(f"Feature vectors created. Total samples: {len(X)}")
    return np.array(X), np.array(y)


# Create and split feature vectors
print("Creating and splitting feature vectors...")
X, y = create_feature_vectors(G, edges)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")

with open("feature_vectors_rf.pkl", "wb") as f:
    pickle.dump((X, y, X_train, X_test, y_train, y_test), f)

print("Feature vectors and splits saved successfully.")

with open("feature_vectors_rf.pkl", "rb") as f:
    X, y, X_train, X_test, y_train, y_test = pickle.load(f)

print("Feature vectors and splits loaded successfully.")
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")



Libraries imported successfully.
Starting graph preprocessing...
Loading original features and edges...
Loaded 30075 nodes with features.
Loaded 701311 edges.
Creating graph from edges...
Graph created with 114324 nodes and 701311 edges.
Identifying the largest connected component...
Largest connected component has 111251 nodes and 699461 edges.
Assigning features to nodes...
Node features assigned.
Graph preprocessing complete. Final graph has 111251 nodes and 699461 edges.
Performing BFS to ensure all nodes are connected...
All nodes are connected. The graph is a single connected component.
Creating and splitting feature vectors...
Creating feature vectors for ML tasks...
Processing positive samples (existing edges)...
Processed 50000 positive samples.
Processed 100000 positive samples.
Processed 150000 positive samples.
Processed 200000 positive samples.
Processed 250000 positive samples.
Processed 300000 positive samples.
Processed 350000 positive samples.
Processed 400000 positive

In [25]:
# Section 3B: Train the Random Forest Model
print("Training the Random Forest model...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Model training complete.")

# Save the trained model
with open("trained_model_rf.pkl", "wb") as f:
    pickle.dump(model, f)
print("Random Forest model saved to 'trained_model_rf.pkl'.")

# Evaluate the model
print("Evaluating the model on the test set...")
test_accuracy = model.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

y_pred = model.predict(X_test)
test_precision = precision_score(y_test, y_pred, zero_division=0)
test_recall = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1: {test_f1:.4f}")

with open("trained_model_rf.pkl", "rb") as f:
    model = pickle.load(f)
print("Random Forest model loaded successfully from 'trained_model_rf.pkl'.")


def find_best_nodes(G, neighbors, target_node):
    """
    Find the best nodes among neighbors based on shortest path distance to the target.
    Returns the list of tied 'best' nodes with the minimum distance.
    """
    distances = {
        neighbor: nx.shortest_path_length(G, source=neighbor, target=target_node)
        for neighbor in neighbors
    }
    min_distance = min(distances.values())
    best_nodes = [node for node, dist in distances.items() if dist == min_distance]
    return best_nodes

def predict_next_node(model, G, current_node, target_node, visited, prediction_cache, debug=False):
    """
    Predict the next node from the current node and return step-by-step accuracy.
    Using Random Forest model with predict_proba.
    """
    if debug:
        print(f"Predicting next node from current node: {current_node}")

    neighbors = [
        neighbor for neighbor in G.neighbors(current_node)
        if "features" in G.nodes[neighbor] and neighbor not in visited
    ]

    if not neighbors:
        random_choice = random.choice(list(G.neighbors(current_node))) if len(list(G.neighbors(current_node))) > 0 else None
        if debug:
            print("Opting for random choice due to no unvisited neighbors with features.")
        return random_choice, False  # No "best choice" available, random node selected
    
    if target_node in neighbors:
        if debug:
            print(f"Target node {target_node} is a direct neighbor. Automatically selecting it.")
        return target_node, True  # Automatically move to the destination and mark as accurate

    best_nodes = find_best_nodes(G, neighbors, target_node)
    
    # Predict probabilities for neighbors
    predictions = []
    for neighbor in neighbors:
        feature_vector = G.nodes[neighbor]["features"] - G.nodes[target_node]["features"]
        prob = model.predict_proba(feature_vector.reshape(1, -1))[0][1]  # Probability of class '1'
        predictions.append((neighbor, prob))

    # Sort by probability (descending)
    predictions.sort(key=lambda x: x[1], reverse=True)

    if debug:
        print(f"Neighbors were available. Highest predicted probability: {predictions[0][1]:.4f}, Selected Node: {predictions[0][0]}")
    chosen_node = predictions[0][0]  # Node with the highest probability

    is_accurate = chosen_node in best_nodes
    return chosen_node, is_accurate

def find_path(model, G, source, target, max_hops=40, debug=False):
    """
    Find the path while tracking accuracy at each step.
    Outputs path and per-move accuracy data.
    """
    if debug:
        print(f"Starting pathfinding from source: {source} to target: {target}, with max hops: {max_hops}")

    current_node = source
    visited = set()
    prediction_cache = {}
    path = [source]  # Track path regardless of success
    hops = 0
    correct_choices = 0
    step_actuals = []  
    step_predictions = []  

    while hops < max_hops:
        if current_node == target:
            if debug:
                print(f"Pathfinding succeeded: target {target} reached.")
            return path, correct_choices, hops, step_actuals, step_predictions

        visited.add(current_node)
        next_node, is_accurate = predict_next_node(model, G, current_node, target, visited, prediction_cache, debug=debug)

        if not next_node:  # No valid moves
            if debug:
                print(f"Pathfinding failed: no valid neighbors from {current_node}.")
            return path, correct_choices, hops, step_actuals, step_predictions

        # A "best node" always exists
        step_actuals.append(1)  
        step_predictions.append(1 if is_accurate else 0)

        if is_accurate:
            correct_choices += 1

        path.append(next_node)
        current_node = next_node
        hops += 1

    # Failed due to max hops
    if debug:
        print(f"Pathfinding failed: exceeded max hops ({max_hops}).")
    return path, correct_choices, hops, step_actuals, step_predictions

def evaluate_pathfinding(model, G, max_hops=20, num_runs=20):
    """
    Evaluate the pathfinding algorithm, calculate metrics, and print paths.
    """
    total_hops = 0
    successful_runs = 0
    total_correct_choices = 0
    total_steps = 0
    all_actuals = []  
    all_predictions = []  

    nodes_with_features = [node for node in G.nodes if "features" in G.nodes[node]]

    for run in range(num_runs):
        if len(nodes_with_features) < 2:
            print("Not enough nodes with features for evaluation.")
            break

        source_node, target_node = random.sample(nodes_with_features, 2)
        print(f"\nRun {run + 1}/{num_runs}: Source {source_node} -> Target {target_node}")

        path, correct_choices, steps, step_actuals, step_predictions = find_path(model, G, source_node, target_node, max_hops)

        # Print path
        print(f"  Path: {path}")

        if path and path[-1] == target_node:
            print(f"  Path found in {steps} steps. Correct choices: {correct_choices}/{steps}")
            successful_runs += 1
            total_hops += steps
        else:
            print(f"  Path failed after {steps} steps.")

        all_actuals.extend(step_actuals)
        all_predictions.extend(step_predictions)

        total_correct_choices += correct_choices
        total_steps += steps

    success_rate = (successful_runs / num_runs) * 100 if num_runs > 0 else 0
    average_hops = total_hops / successful_runs if successful_runs > 0 else float('inf')
    accuracy = total_correct_choices / total_steps if total_steps > 0 else 0.0
    precision = precision_score(all_actuals, all_predictions, zero_division=0)
    recall = recall_score(all_actuals, all_predictions, zero_division=0)
    f1 = f1_score(all_actuals, all_predictions, zero_division=0)

    # Print summary
    print(f"\n--- Summary ---")
    print(f"Success rate: {success_rate:.2f}% ({successful_runs}/{num_runs})")
    print(f"Average hops: {average_hops:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-score: {f1:.2f}")

    return success_rate, average_hops, accuracy, precision, recall, f1



Training the Random Forest model...
Model training complete.
Random Forest model saved to 'trained_model_rf.pkl'.
Evaluating the model on the test set...
Test Accuracy: 0.9910
Test Precision: 0.0000
Test Recall: 0.0000
Test F1: 0.0000
Random Forest model loaded successfully from 'trained_model_rf.pkl'.


In [26]:
# Evaluate pathfinding performance with the Random Forest model
success_rate, average_hops, accuracy, precision, recall, f1 = evaluate_pathfinding(model, G, max_hops=20, num_runs=1000)



Run 1/1000: Source 18071 -> Target 18840
  Path: ['18071', '10034', '1268', '14382', '9978', '18840']
  Path found in 5 steps. Correct choices: 3/5

Run 2/1000: Source 16086 -> Target 24170
  Path: ['16086', '13455', '9640', '10590', '616', '6158', '4074', '540', '17476', '10428', '13645', '13367', '2001', '11290', '11891', '5471', '5711', '12385', '13362', '11712', '13944']
  Path failed after 20 steps.

Run 3/1000: Source 3360 -> Target 16921
  Path: ['3360', '854', '5975', '1258', '15253', '9301', '13600', '5851', '10509', '20214', '29857', '10067', '16256', '17087', '7078', '8557', '19235', '3715', '3178', '2068', '1374']
  Path failed after 20 steps.

Run 4/1000: Source 14860 -> Target 21530
  Path: ['14860', '9640', '13181', '17152', '9807', '13492', '10590', '11590', '17290', '28162', '13143', '15756', '16997', '12851', '14314', '10674', '5363', '11507', '11611', '11507', '11350']
  Path failed after 20 steps.

Run 5/1000: Source 29941 -> Target 23021
  Path: ['29941', '5975', 

  Path: ['5912', '1324', '2944', '16510', '12718', '16510', '6051', '210707', '456859', '15196', '12865', '17290', '16558', '10994', '15558', '19392', '14161', '14860', '11404', '11904', '11875']
  Path failed after 20 steps.

Run 35/1000: Source 14188 -> Target 20058
  Path: ['14188', '173400', '14188', '86659', '5975', '545', '10464', '29996', '21531', '953', '2667', '541', '178', '14571', '8524', '5009', '1872', '5009', '13268', '7543', '15683']
  Path failed after 20 steps.

Run 36/1000: Source 6383 -> Target 27595
  Path: ['6383', '6114', '14249', '4927', '476', '1076', '337', '541', '758', '8683', '534', '21661', '29998', '10428', '20640', '7285', '3131', '1625', '4608', '1605', '18741']
  Path failed after 20 steps.

Run 37/1000: Source 25503 -> Target 28534
  Path: ['25503', '16182', '9858', '5334', '3346', '4692', '26360', '9978', '3849', '9060', '26330', '12434', '16396', '4907', '11123', '18840', '27648', '12428', '16462', '11150', '12806']
  Path failed after 20 steps.

Run

  Path: ['8920', '4709', '19654', '254768', '476204', '259517', '7279', '6921', '832', '16637', '2079', '78', '17855', '14571', '18011', '20842', '21102', '22875', '106496', '8550', '7970']
  Path failed after 20 steps.

Run 69/1000: Source 17421 -> Target 9072
  Path: ['17421', '9640', '5363', '10853', '13845', '12260', '10994', '11071', '23395', '14314', '17290', '12144', '10383', '8410', '12929', '12865', '4008', '10648', '10590', '10711', '9935']
  Path failed after 20 steps.

Run 70/1000: Source 26916 -> Target 13716
  Path: ['26916', '25964', '541', '8240', '540', '4074', '5975', '1057', '4284', '535', '4024', '1904', '29754', '15474', '15656', '884', '21914', '26622', '12878', '19587', '6629']
  Path failed after 20 steps.

Run 71/1000: Source 11677 -> Target 27208
  Path: ['11677', '10662', '12101', '95379', '10662', '11654', '10662', '789092', '11654', '595676', '914830', '12713', '11985', '9640', '17152', '13181', '15558', '10994', '12851', '14314', '13469']
  Path failed aft

  Path: ['9643', '20109', '12434', '3853', '3346', '9858', '18563', '4907', '20644', '10428', '6595', '22800', '12026', '13340', '4608', '1605', '18741', '25007', '14249', '6114', '4957']
  Path failed after 20 steps.

Run 102/1000: Source 6114 -> Target 2541
  Path: ['6114', '4957', '4689', '3859', '14571', '9888', '6227', '2025', '5975', '18567', '26511', '18567', '28272', '4913', '3514', '11378', '12699', '540', '8169', '22388', '348']
  Path failed after 20 steps.

Run 103/1000: Source 18237 -> Target 26221
  Path: ['18237', '89966', '119920', '2498', '35979', '2498', '219292', '6157', '9528', '5851', '2783', '21610', '13033', '12852', '20276', '127683', '3650', '167711', '220474', '16618', '20214']
  Path failed after 20 steps.

Run 104/1000: Source 16621 -> Target 13333
  Path: ['16621', '15105', '712352', '15105', '1173497', '14172', '29754', '21305', '6629', '19587', '12878', '5666', '26622', '13541', '3055', '1904', '25536', '16411', '6536', '1830', '14964']
  Path failed afte

  Path: ['21305', '25626', '6536', '13541', '16411', '25536', '1904', '4024', '15537', '5703', '14571', '21382', '5975', '18608', '20053', '24572', '541', '545', '22388', '11859', '348']
  Path failed after 20 steps.

Run 134/1000: Source 10551 -> Target 20369
  Path: ['10551', '12852', '6681', '13944', '11712', '5471', '10823', '2001', '12385', '11891', '18110', '5711', '13842', '13766', '10837', '21001', '22158', '13178', '20372', '19535', '1264']
  Path failed after 20 steps.

Run 135/1000: Source 13268 -> Target 17792
  Path: ['13268', '5975', '17476', '4448', '17792']
  Path found in 4 steps. Correct choices: 3/4

Run 136/1000: Source 18245 -> Target 26774
  Path: ['18245', '15824', '2001', '15253', '10428', '27807', '18196', '14434', '407', '18741', '1605', '2998', '13680', '6541', '4608', '20077', '17884', '22800', '7934', '817414', '28537']
  Path failed after 20 steps.

Run 137/1000: Source 24551 -> Target 5086
  Path: ['24551', '921352', '609233', '24551', '2161054', '294045'

...

  Path: ['21875', '25018', '6034', '20793', '6277', '4786', '15253', '5768', '20053', '24572', '28474', '9888', '14571', '4024', '4877', '18840', '16462', '23747', '9297', '801', '11717']
  Path failed after 20 steps.

Run 863/1000: Source 5940 -> Target 2900
  Path: ['5940', '1222', '7970', '3275', '3592', '8550', '348', '22388', '483', '854', '5009', '4448', '8678', '27995', '721', '17476', '27544', '12780', '29889', '26403', '24720']
  Path failed after 20 steps.

Run 864/1000: Source 4547 -> Target 26526
  Path: ['4547', '12385', '2001', '21179', '6034', '6571', '6690', '25716', '10571', '9874', '3859', '5975', '10464', '20053', '24572', '26526']
  Path found in 15 steps. Correct choices: 6/15

Run 865/1000: Source 26473 -> Target 27103
  Path: ['26473', '3339', '28974', '3507', '211718', '12434', '20902', '5808', '1324', '28874', '936134', '800433', '15173', '9786', '19880', '22136', '17900', '16510', '24170', '18624', '1058244']
  Path failed after 20 steps.

Run 866/1000: Source

  Path: ['26887', '19276', '389753', '19276', '108601', '65850', '14571', '14054', '18567', '5975', '7199', '5754', '541', '483', '721', '27995', '9888', '13367', '2006', '535', '4024']
  Path failed after 20 steps.

Run 896/1000: Source 19354 -> Target 16591
  Path: ['19354', '3944', '541', '19631', '29996', '5975', '17476', '4448', '18311', '7199', '2969', '23668', '21170', '757798', '18567', '16638', '25679', '15041', '28465', '4390', '1057']
  Path failed after 20 steps.

Run 897/1000: Source 25626 -> Target 12267
  Path: ['25626', '6536', '18818', '3055', '21340', '29754', '17077', '6629', '4024', '14571', '14054', '18567', '5975', '7199', '5754', '541', '483', '721', '27995', '9888', '13367']
  Path failed after 20 steps.

Run 898/1000: Source 25085 -> Target 2068
  Path: ['25085', '13091', '12492', '17792', '4448', '14571', '21382', '5975', '13258', '23535', '4390', '21661', '29998', '27526', '541', '23906', '28272', '23402', '4826', '21531', '953']
  Path failed after 20 steps.

  Path: ['16867', '78431', '16150', '18378', '26622', '16411', '6536', '25626', '21305', '29754', '15474', '18818', '1910', '1904', '541', '758', '19487', '1222', '29291', '581', '28729']
  Path failed after 20 steps.

Run 929/1000: Source 14873 -> Target 13520
  Path: ['14873', '14296', '401853', '11970', '11087', '12796', '10625', '13520']
  Path found in 7 steps. Correct choices: 4/7

Run 930/1000: Source 6702 -> Target 23823
  Path: ['6702', '27336', '6928', '3171', '1323', '1258', '5975', '13258', '23535', '14571', '9888', '28474', '24572', '528', '19943', '15465', '3944', '2079', '4550', '14249', '1498']
  Path failed after 20 steps.

Run 931/1000: Source 15333 -> Target 4556
  Path: ['15333', '157401', '21093', '9858', '8773', '5009', '5975', '21382', '9925', '12699', '7199', '24320', '23912', '24320', '1399', '11712', '4199', '11290', '12385', '6597', '7379']
  Path failed after 20 steps.

Run 932/1000: Source 14366 -> Target 15656
  Path: ['14366', '12823', '10994', '14517', '

  Path: ['17087', '6891', '6928', '17919', '12062', '12434', '20109', '26306', '1536', '9858', '23546', '12786', '23157', '696365', '602747', '734803', '10034', '28916', '11717', '24303', '9897']
  Path failed after 20 steps.

Run 961/1000: Source 20979 -> Target 23149
  Path: ['20979', '41864', '483', '5975', '9925', '9039', '541', '4857', '4826', '866', '25085', '14571', '23535', '13258', '8684', '2532', '14434', '4608', '2799', '22800', '6595']
  Path failed after 20 steps.

Run 962/1000: Source 2366 -> Target 15555
  Path: ['2366', '3241', '16637', '178', '14571', '15555']
  Path found in 5 steps. Correct choices: 4/5

Run 963/1000: Source 13393 -> Target 7552
  Path: ['13393', '8420', '3159', '5588', '4957', '4689', '4448', '14571', '6041', '14571', '17855', '10464', '5975', '5987', '3360', '854', '5009', '540', '3944', '2079', '17769']
  Path failed after 20 steps.

Run 964/1000: Source 5890 -> Target 6571
  Path: ['5890', '5615', '15667', '12434', '9060', '3849', '3171', '3306',

  Path: ['12233', '10648', '10590', '13492', '14119', '14860', '9640', '12847', '25292']
  Path found in 8 steps. Correct choices: 5/8

Run 995/1000: Source 7725 -> Target 26619
  Path: ['7725', '25833', '24226', '6941', '4106', '4368', '24590', '10040', '7088', '11893', '26140', '28664', '24395', '9805', '25582', '25113', '27849', '28705', '190', '28895', '59440']
  Path failed after 20 steps.

Run 996/1000: Source 26737 -> Target 28106
  Path: ['26737', '5768', '7729', '16128', '13395', '18311', '6277', '6034', '18220', '4786', '7970', '20186', '13178', '19247', '14750', '19535', '1264', '8524', '3339', '22951', '20839']
  Path failed after 20 steps.

Run 997/1000: Source 8225 -> Target 7122
  Path: ['8225', '6034', '6690', '6347', '6106', '17026', '10478', '20037', '25018', '20584', '3812', '22961', '4786', '21948', '50267', '3754', '6158', '9858', '10266', '649', '12497']
  Path failed after 20 steps.

Run 998/1000: Source 11084 -> Target 23402
  Path: ['11084', '2001', '21179', '6

In [28]:
#RANDOM FOREST
print(success_rate, average_hops, accuracy, precision, recall, f1)

15.1 8.344370860927153 0.42461622807017546 1.0 0.42461622807017546 0.5961131421974216


In [27]:
print("Feature Importances:")
for i, importance in enumerate(model.feature_importances_):
    print(f"Feature {i}: {importance:.4f}")

Feature Importances:
Feature 0: 0.1988
Feature 1: 0.2017
Feature 2: 0.2028
Feature 3: 0.0155
Feature 4: 0.0111
Feature 5: 0.0196
Feature 6: 0.1741
Feature 7: 0.1764
